## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/cesarfernandez/Desktop/Analysis_Projects/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bengkulu,ID,-3.8004,102.2655,78.64,85,100,3.13,light rain
1,1,Honiara,SB,-9.4333,159.9500,78.06,87,59,5.79,broken clouds
2,2,Cape Town,ZA,-33.9258,18.4232,73.09,49,81,9.26,broken clouds
3,3,Teloloapan,MX,18.3500,-99.8500,63.66,84,61,0.96,broken clouds
4,4,Victoria,HK,22.2855,114.1577,77.32,78,97,1.01,light rain
5,5,Tuatapere,NZ,-46.1333,167.6833,39.58,87,46,4.68,scattered clouds
6,6,Jinchang,CN,38.4953,102.1739,41.76,42,59,1.59,broken clouds
7,7,Punta Arenas,CL,-53.1500,-70.9167,48.31,57,0,12.66,clear sky
8,8,Hermanus,ZA,-34.4187,19.2345,70.90,85,100,10.31,overcast clouds
9,9,Airai,TL,-8.9266,125.4092,62.22,89,93,1.28,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bengkulu,ID,-3.8004,102.2655,78.64,85,100,3.13,light rain
1,1,Honiara,SB,-9.4333,159.9500,78.06,87,59,5.79,broken clouds
2,2,Cape Town,ZA,-33.9258,18.4232,73.09,49,81,9.26,broken clouds
4,4,Victoria,HK,22.2855,114.1577,77.32,78,97,1.01,light rain
8,8,Hermanus,ZA,-34.4187,19.2345,70.90,85,100,10.31,overcast clouds
13,13,Meulaboh,ID,4.1363,96.1285,77.81,86,100,2.30,overcast clouds
16,16,Faanui,PF,-16.4833,-151.7500,78.24,67,100,14.90,overcast clouds
21,21,Mahebourg,MU,-20.4081,57.7000,75.25,72,33,5.99,scattered clouds
23,23,Touros,BR,-5.1989,-35.4608,86.90,60,36,15.73,scattered clouds
25,25,Christiana,ZA,-27.9140,25.1611,81.73,15,0,3.27,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                250
City                   250
Country                250
Lat                    250
Lng                    250
Max Temp               250
Humidity               250
Cloudiness             250
Wind Speed             250
Current Description    250
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bengkulu,ID,78.64,light rain,-3.8004,102.2655,
1,Honiara,SB,78.06,broken clouds,-9.4333,159.9500,
2,Cape Town,ZA,73.09,broken clouds,-33.9258,18.4232,
4,Victoria,HK,77.32,light rain,22.2855,114.1577,
8,Hermanus,ZA,70.90,overcast clouds,-34.4187,19.2345,
13,Meulaboh,ID,77.81,overcast clouds,4.1363,96.1285,
16,Faanui,PF,78.24,overcast clouds,-16.4833,-151.7500,
21,Mahebourg,MU,75.25,scattered clouds,-20.4081,57.7000,
23,Touros,BR,86.90,scattered clouds,-5.1989,-35.4608,
25,Christiana,ZA,81.73,clear sky,-27.9140,25.1611,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
new_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
new_hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bengkulu,ID,78.64,light rain,-3.8004,102.2655,Grage Hotel Bengkulu
1,Honiara,SB,78.06,broken clouds,-9.4333,159.9500,Heritage Park Hotel
2,Cape Town,ZA,73.09,broken clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town
4,Victoria,HK,77.32,light rain,22.2855,114.1577,Mini Hotel Central
8,Hermanus,ZA,70.90,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
13,Meulaboh,ID,77.81,overcast clouds,4.1363,96.1285,Kost Pria
16,Faanui,PF,78.24,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
21,Mahebourg,MU,75.25,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
23,Touros,BR,86.90,scattered clouds,-5.1989,-35.4608,INN NEW HORIZON
25,Christiana,ZA,81.73,clear sky,-27.9140,25.1611,At-Neli Guesthouse


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
new_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in new_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = new_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = new_hotel_df[["Lat", "Lng"]]
max_temp = new_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))